In [6]:
# setups

import os
import json

import boto3
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role

from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker.predictor import Predictor

In [2]:
session = sagemaker.Session()
bucket = session.default_bucket()
prefix = "sagemaker/generative-ai-controlnet"
role = get_execution_role()

uncomment the below code cells to prepare model artifacts and upload to s3 if you haven't done so

In [ ]:
import tarfile
from pathlib import Path
import tqdm
def create_tar(tarfile_name: str, local_path: Path):
    """
    Create a tar.gz archive with the content of `local_path`.
    """
    file_list = [k for k in local_path.glob("**/*.*") if f"{k.relative_to(local_path)}"[0] != "."]
    pbar = tqdm.tqdm(file_list, unit="files")
    with tarfile.open(tarfile_name, mode="w:gz") as archive:
        for k in pbar:
            pbar.set_description(f"{k}")
            archive.add(k, arcname=f"{k.relative_to(local_path)}")
    tar_size = Path(tarfile_name).stat().st_size / 10**6
    return 

In [ ]:
%%time
tar_file = "model.tar.gz"
tar_size = create_tar(tar_file, Path("../models"))
print(f"Created {tar_file}")

In [ ]:
model_data_path = s3_path_join("s3://", bucket, prefix + "/models")
print(f"Uploading Models to {model_data_path}")
model_uri = S3Uploader.upload("model.tar.gz", model_data_path)
print(f"Uploaded roberta model to {model_uri}")

In [13]:
model_uri = 's3://sagemaker-us-east-1-631450739534/sagemaker/generative-ai-controlnet/models/model.tar.gz'

In [16]:
model = PyTorchModel(
    entry_point="inference.py",
    source_dir="ControlNet",
    role=role,
    model_data=model_uri,
    framework_version="1.12.1",
    py_version="py38",
    predictor_cls=Predictor,
    code_location=f's3://{bucket}/{prefix}/sourcedir',
    model_server_workers=1,
    
)

In [ ]:
# set local_mode to False if you want to deploy on a remote
# SageMaker instance

local_mode = True

if local_mode:
    instance_type = "local"
else:
    instance_type = "ml.g5.xlarge"

predictor = model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

In [18]:
file_name = "../data.json"
with open(file_name, 'r') as file:
    json_data = json.load(file)
    


In [ ]:
predictor.predict(json_data)

In [24]:
predictor.delete_model()
predictor.delete_endpoint()